### Installation
Install the packages required for executing this notebook.

In [10]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [1]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

: 

Check the versions of the packages you installed. The KFP SDK version should be >=2

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'kfp'
google-cloud-aiplatform==1.70.0
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'google_cloud_pipeline_components'


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip

ModuleNotFoundError: No module named 'kfp'

#### Pipeline Configurations

In [9]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2024-436414"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2024_17"

#### Create the Components from Component Specifications 

In [10]:
@dsl.container_component
def data_ingestion(project: str, bucket: str, data_file_name: str,  features: Output[Artifact]): 

    return dsl.ContainerSpec(
        image='us-central1-docker.pkg.dev/your project id/labrepo/dataingestor:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id',project,'--bucket',bucket,'--file_name',data_file_name,'--feature_path', features.path])

In [11]:
@dsl.container_component
def mlp_training(project: str, features: Input[Artifact], model_bucket: str,  metrics: OutputPath(str)):
    
    return dsl.ContainerSpec(
        image='us-central1-docker.pkg.dev/your project id/labrepo/mlptrainer:0.0.1',
        command=[
            'python3', '/pipelines/component/src/component.py'
        ],
        args=['--project_id',project,'--feature_path',features.path,'--model_repo',model_bucket,'--metrics_path', metrics])

#### Define the Pipeline

In [6]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="student-predictor-mlp")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str):
    
    # The first step    
    di_op = data_ingestion(
        project=project_id,
        bucket=data_bucket,
        data_file_name=trainset_filename
    )

    # The second step 
    training_op = mlp_training(
        project=project_id,
        model_bucket=model_repo,       
        features=di_op.outputs['features']
    )

#### Compile the Pipeline

In [7]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='student_predictor_mlp.yaml')

#### Run the Pipeline

In [8]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)


job = aip.PipelineJob(
    display_name="student-predictor-mlp-pipeline",
    template_path="student_predictor_mlp.yaml",
    enable_caching=False,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'project_id': PROJECT_ID,
        'data_bucket': 'data_de2024_17', # makesure to use your data bucket name 
        'trainset_filename': 'training_set.csv',
        'model_repo':'models_de2024_17' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/966204339179/locations/us-central1/pipelineJobs/diabetes-predictor-mlp-20241008202233
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/966204339179/locations/us-central1/pipelineJobs/diabetes-predictor-mlp-20241008202233')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/diabetes-predictor-mlp-20241008202233?project=966204339179
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-predictor-mlp-20241008202233 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-predictor-mlp-20241008202233 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-predictor-mlp-20241008202233 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/96620433